# **Scraping Brent Indicator Data**

Data for Urals and Dutch TFF could not be obtained for scraping due to restrictions and use of CDN on the few available sources. Similar methodology to this can however be adopted for both indicators where data is readily available

In [76]:
# importing the necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re 

In [77]:
# Request to website and download HTML contents
url='https://markets.ft.com/data/etfs/tearsheet/historical?s=BNQA:GER:EUR'
req=requests.get(url)
content=req.text

In [78]:
#viewing the scraped data
content

'\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="en-us" class="no-js core o-hoverable-on">\r\n<head>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n    <meta charset="utf-8" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\r\n    <meta name="description" content="Latest ICE BRENT CRUDE OIL ETC (BNQA:GER:EUR) share price with interactive charts, historical prices, comparative analysis, forecasts, business profile and more." />\r\n    <meta name="keywords" content="ICE BRENT CRUDE OIL ETC price, ICE BRENT CRUDE OIL ETC quote" />\r\n    <link rel="shortcut icon" href="//im.ft-static.com/m/icons/favicon.ico" type="image/x-icon" />\r\n    \r\n    <title>ICE BRENT CRUDE OIL ETC, BNQA:GER:EUR historical prices - FT.com</title>\r\n    <!--[if lt IE 9]>\r\n        <script src="/data/content/scripts/vendor/html5shiv.js"></script>\r\n    <![endif]-->\r\n    <link rel="stylesheet" href="/data/content/styles/origami/origami.css?v=2" />\r\n    <link hr

In [79]:
#parsing the data using beautiful soup for easily readability and to enable us find the html tag of the desired data
content=BeautifulSoup(content, 'html.parser')

In [80]:
#extracting the data using the table's class and formatting appropriately after inspection
rows = content.find_all('table', class_= 'mod-ui-table mod-tearsheet-historical-prices__results mod-ui-table--freeze-pane')
rows = rows.pop(0)
rows = rows.text.strip('[]')

In [81]:
#viewing the resulting output 
rows

'DateOpenHighLowCloseVolumeWednesday, June 07, 2023Wed, Jun 07, 202384.4684.4684.4684.46----Tuesday, June 06, 2023Tue, Jun 06, 202383.3683.3683.3683.36----Monday, June 05, 2023Mon, Jun 05, 202383.8083.8083.8083.80----Friday, June 02, 2023Fri, Jun 02, 202382.0782.0782.0782.07----Thursday, June 01, 2023Thu, Jun 01, 202380.9680.9680.9680.96----Wednesday, May 31, 2023Wed, May 31, 202380.3380.3380.3380.33----Tuesday, May 30, 2023Tue, May 30, 202380.7380.7380.7380.73----Monday, May 29, 2023Mon, May 29, 202382.9082.9082.9082.90----Friday, May 26, 2023Fri, May 26, 202382.9082.9082.9082.90----Thursday, May 25, 2023Thu, May 25, 202382.9082.9082.9082.90----Wednesday, May 24, 2023Wed, May 24, 202384.5584.5584.5584.55----Tuesday, May 23, 2023Tue, May 23, 202383.3383.3383.3383.33----Monday, May 22, 2023Mon, May 22, 202381.9681.9681.9681.96----Friday, May 19, 2023Fri, May 19, 202380.9580.9580.9580.95----Thursday, May 18, 2023Thu, May 18, 202382.0682.0682.0682.06----Wednesday, May 17, 2023Wed, May 17,

# **Obtaining Data In Standardized Format**

In [82]:
#removing clutter
rows = rows.replace('DateOpenHighLowCloseVolume', '')

# Split the remaining text by '----' to obtain each day's data as an item in a list
rows = rows.split('----')


#defining a regular expression to extract the required data into a dataframe 
def process_text(text):
    pattern = r"([A-Za-z]+\s\d{2},\s\d{4}).*?\d{2}(\d+\.\d{2})(\d+\.\d{2})(\d+\.\d{2})(\d+\.\d{2})"

    data = []

    # Iterate over each row in the text
    for row in text.split('\n'):
        # Skip empty rows
        if not row:
            continue

        # Apply the regular expression pattern
        matches = re.findall(pattern, row)

        # Extract the matched data and append it to the list
        for match in matches:
            data.append((match[0], match[1], match[2], match[3], match[4]))

    # Create a dataframe from the data list
    df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close"])

    # Correct the open column by removing the first two digits
    df["Open"] = df["Open"].str[2:]

    return df



# Process each text in the list
result_dfs = []
for row in rows:
    result_df = process_text(row)
    result_dfs.append(result_df)

# Concatenate all dataframes into a single dataframe
brentpricedata = pd.concat(result_dfs, ignore_index=True)
brentpricedata

,Date,Open,High,Low,Close
0,"June 07, 2023",84.46,84.46,84.46,84.46
1,"June 06, 2023",83.36,83.36,83.36,83.36
2,"June 05, 2023",83.80,83.80,83.80,83.80
3,"June 02, 2023",82.07,82.07,82.07,82.07
4,"June 01, 2023",80.96,80.96,80.96,80.96
5,"May 31, 2023",80.33,80.33,80.33,80.33
6,"May 30, 2023",80.73,80.73,80.73,80.73
7,"May 29, 2023",82.90,82.90,82.90,82.90
8,"May 26, 2023",82.90,82.90,82.90,82.90
9,"May 25, 2023",82.90,82.90,82.90,82.90
